In [0]:
from pyspark.sql.functions import date_add, concat_ws, concat, initcap, upper, lower, col, when, count, year, substring, desc, date_format, current_timestamp
from pyspark.sql.window import Window
from pyspark.sql import Row
from pyspark.sql.types import StringType
from datetime import *

In [0]:
#Dados do Motor de Crédito
df_CreditoMotor = spark.sql("SELECT MT.Pedido, SUM(MT.valor_do_credito) AS ValorEstorno FROM app_atendimento.tb_motor_sap MT WHERE MT.Direcionamento IN ('Confirmado', 'Disponivel ao Cliente', 'Disponivel Cliente') GROUP BY MT.Pedido")

df_Estorno = spark.sql("SELECT MT.Pedido, SUM(MT.valor_do_credito) AS Estorno FROM app_atendimento.tb_motor_sap MT WHERE MT.Direcionamento IN ('Confirmado', 'Disponivel ao Cliente', 'Disponivel Cliente') GROUP BY MT.Pedido")

df_Entrega_Confirmada = spark.sql("SELECT Entrega Entrega_Confirmada FROM app_atendimento.tb_motor_sap M WHERE M.Direcionamento IN ('Confirmado', 'Disponivel ao Cliente', 'Disponivel Cliente')")

#Dados do Front
df_Compra = spark.table("context_site.Compra").select(col("IdCompra"), "IdUnidadeNegocio", "bandeira", col("IdUnidadeNegocio").alias("Loja"), col("IdUnidadeNegocio").alias("Id")) 

df_CompraEntrega = spark.table("context_site.CompraEntrega").select(col("IdCompra"), "IdCompraEntrega", "GerencialId", "bandeira",col("IdCompra").alias("Pedido_Front"))

df_CompraEntregaStatusEstorno = spark.table("context_site.CompraEntregaStatusEstorno").select(col("IdCompra"), "DataStatusSolicitacao", "bandeira", "dataultimaatualizacao", "Status").filter(col("DataUltimaatuAlizacao") >= date_add(current_timestamp(),-90))

df_CompraCliente = spark.table("context_site.CompraCliente").select("IdCompra", "Nome", "Sobrenome", "Email", "TelefoneResidencial", "TelefoneCelular", "bandeira").filter(col("TipoCliente") == "0")

df_CompraEntregaEstorno = spark.table("context_site.CompraEntregaEstorno").select("IdCompraEntregaNova", "Valor", "TipoPagamento", "bandeira").filter(col("TipoPagamento").isin("0", "1", "2", "3"))

df_CompraFormaPagamento = spark.table("context_site.CompraFormaPagamento").select("IdCompra", "ValorComJuros", "IdFormaPagamento", "IdCompraFormaPagamentoStatus", "bandeira").filter(col("IdCompraFormaPagamentoStatus") == "2")

df_FormaPagamento = spark.table("context_site.FormaPagamento").select("IdFormaPagamento", "bandeira", col("Nome").alias("Nome_Cartao"))

df_TipoPagamento = spark.table("context_site.TipoPagamento").select("IdTipoPagamento" ,col("Nome").alias("Tipo_Estorno")).filter(col("IdTipoPagamento").isin("0", "1", "2", "3"))

df_PagamentoCliente = spark.sql("SELECT CF.IdCompra, SUM(CF.ValorComJuros) AS ValorPagoPeloCliente FROM context_site.CompraFormaPagamento CF GROUP BY CF.IdCompra")

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-3776382013132954> in <module> 
 1 #Dados do Motor de Crédito 
 ----> 2 df_CreditoMotor = spark . sql ( "SELECT MT.Pedido, SUM(MT.valor_do_credito) AS ValorEstorno FROM app_atendimento.tb_motor_sap MT WHERE MT.Direcionamento IN ('Confirmado', 'Disponivel ao Cliente', 'Disponivel Cliente') GROUP BY MT.Pedido" ) 
 3 
 4 df_Estorno = spark . sql ( "SELECT MT.Pedido, SUM(MT.valor_do_credito) AS Estorno FROM app_atendimento.tb_motor_sap MT WHERE MT.Direcionamento IN ('Confirmado', 'Disponivel ao Cliente', 'Disponivel Cliente') GROUP BY MT.Pedido" ) 
 5 

 /databricks/spark/python/pyspark/sql/session.py in sql (self, sqlQuery) 
 775 [ Row ( f1 = 1 , f2 = 'row1' ) , Row ( f1 = 2 , f2 = 'row2' ) , Row ( f1 = 3 , f2 = 'row3' ) ] 
 776 """
 --> 777 return DataFrame ( self . _jsparkSession . sql ( sqlQuery ) , self . _wrapped ) 
 778 
 779 def table ( self , tableName ) : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 114 # Hide where the exception came from that shows a non-Pythonic 
 115 # JVM exception message. 
 --> 116 raise converted from None 
 117 else : 
 118 raise 

 AnalysisException : Table or view not found: app_atendimento.tb_motor_sap; line 1 pos 64;
'Aggregate ['MT.Pedido], ['MT.Pedido, 'SUM('MT.valor_do_credito) AS ValorEstorno#41]
+- 'Filter 'MT.Direcionamento IN (Confirmado,Disponivel ao Cliente,Disponivel Cliente)
 +- 'SubqueryAlias MT
 +- 'UnresolvedRelation [app_atendimento, tb_motor_sap], [], false

In [0]:
df_Front = (df_Compra.join(df_CompraEntrega, ['IdCompra'], how='inner')
                     .join(df_CompraEntregaStatusEstorno, (df_CompraEntregaStatusEstorno.IdCompra == df_Compra.IdCompra) &
                          (df_CompraEntregaStatusEstorno.bandeira == df_Compra.bandeira), how='inner')
                     .join(df_CompraCliente, (df_CompraCliente.IdCompra == df_Compra.IdCompra) &
                          (df_CompraCliente.bandeira == df_Compra.bandeira), how='inner')
                     .join(df_CompraFormaPagamento, (df_CompraFormaPagamento.IdCompra == df_Compra.IdCompra) &
                          (df_CompraFormaPagamento.bandeira == df_Compra.bandeira), how='inner')
                     .join(df_CompraEntregaEstorno, (df_CompraEntregaEstorno.IdCompraEntregaNova == df_CompraEntrega.IdCompraEntrega) &
                          (df_CompraEntregaEstorno.bandeira == df_CompraEntrega.bandeira), how='inner')  
                     .join(df_FormaPagamento, (df_FormaPagamento.IdFormaPagamento == df_CompraFormaPagamento.IdFormaPagamento), how='inner')
                     .join(df_TipoPagamento, (df_TipoPagamento.IdTipoPagamento == df_CompraEntregaEstorno.TipoPagamento), how='inner')                     
                     .join(df_CreditoMotor, (df_CreditoMotor.Pedido == df_Compra.IdCompra), how='inner')
                     .join(df_PagamentoCliente, (df_PagamentoCliente.IdCompra == df_Compra.IdCompra), how='inner')
                     .join(df_Entrega_Confirmada, (df_Entrega_Confirmada.Entrega_Confirmada == df_CompraEntrega.GerencialId), how='inner')  
                     .join(df_Estorno,(df_Estorno.Pedido == df_Compra.IdCompra),how='inner')
                     .withColumn("Nome", concat_ws(' ',initcap(col("Nome")), initcap(col("Sobrenome"))))
                     .withColumn("Email", lower(col("Email"))) 
                     .withColumn("Valor", (col("Valor") * - 1))                     
                     .withColumn("Loja", when(col("Loja").isin(1, 2),  "PF")
                                        .when(col("Loja").isin(5, 6),  "EX")
                                        .when(col("Loja").isin(8, 11), "CB"))
                     .withColumn("Id",   when(col("Id").isin(1, 2),  concat(col("Loja"), col("Pedido_Front")))
                                        .when(col("Id").isin(5, 6),  concat(col("Loja"), col("Pedido_Front")))
                                        .when(col("Id").isin(8, 11), concat(col("Loja"), col("Pedido_Front"))))
                     .withColumn("ValorEstorno", when(col("ValorEstorno") == col("ValorPagoPeloCliente"), "Total").otherwise("Parcial"))
                     .drop(df_CompraEntregaStatusEstorno.IdCompra)
                     .drop(df_CompraFormaPagamento.IdCompra)
                     .drop(df_CompraFormaPagamento.IdFormaPagamento)
                     .drop(df_PagamentoCliente.IdCompra)
                     .drop(df_CompraCliente.IdCompra)
                     .drop(df_Estorno.Pedido)
                     .drop(df_Compra.IdCompra)
                     .select("Id", "Nome", "Email", "TelefoneResidencial", "TelefoneCelular", "Loja", "Estorno", "Nome_Cartao", "Tipo_Estorno", "Pedido", col("ValorEstorno").alias("Restituicao"), "dataultimaatualizacao").filter(col("IdTipoPagamento").isin("0", "1", "3")))

In [0]:
df_Vale = (df_Compra.join(df_CompraEntrega, ['IdCompra'], how='inner')
                     .join(df_CompraEntregaStatusEstorno, (df_CompraEntregaStatusEstorno.IdCompra == df_Compra.IdCompra) &
                          (df_CompraEntregaStatusEstorno.bandeira == df_Compra.bandeira), how='inner')
                     .join(df_CompraCliente, (df_CompraCliente.IdCompra == df_Compra.IdCompra) &
                          (df_CompraCliente.bandeira == df_Compra.bandeira), how='inner')
                     .join(df_CompraFormaPagamento, (df_CompraFormaPagamento.IdCompra == df_Compra.IdCompra) &
                          (df_CompraFormaPagamento.bandeira == df_Compra.bandeira), how='inner')
                     .join(df_CompraEntregaEstorno, (df_CompraEntregaEstorno.IdCompraEntregaNova == df_CompraEntrega.IdCompraEntrega) &
                          (df_CompraEntregaEstorno.bandeira == df_CompraEntrega.bandeira), how='inner')  
                     .join(df_FormaPagamento, (df_FormaPagamento.IdFormaPagamento == df_CompraFormaPagamento.IdFormaPagamento), how='inner')
                     .join(df_TipoPagamento, (df_TipoPagamento.IdTipoPagamento == df_CompraEntregaEstorno.TipoPagamento), how='inner')                     
                     .join(df_CreditoMotor, (df_CreditoMotor.Pedido == df_Compra.IdCompra), how='inner')
                     .join(df_PagamentoCliente, (df_PagamentoCliente.IdCompra == df_Compra.IdCompra), how='inner')
                     .join(df_Entrega_Confirmada, (df_Entrega_Confirmada.Entrega_Confirmada == df_CompraEntrega.GerencialId), how='inner')
                     .join(df_Estorno,(df_Estorno.Pedido == df_Compra.IdCompra),how='inner')
                     .withColumn("Nome", concat_ws(' ',initcap(col("Nome")), initcap(col("Sobrenome"))))
                     .withColumn("Email", lower(col("Email"))) 
                     .withColumn("Valor", (col("Valor") * - 1))                     
                     .withColumn("Loja", when(col("Loja").isin(1, 2),  "PF")
                                        .when(col("Loja").isin(5, 6),  "EX")
                                        .when(col("Loja").isin(8, 11), "CB"))
                     .withColumn("Id",   when(col("Id").isin(1, 2),  concat(col("Loja"), col("Pedido_Front")))
                                        .when(col("Id").isin(5, 6),  concat(col("Loja"), col("Pedido_Front")))
                                        .when(col("Id").isin(8, 11), concat(col("Loja"), col("Pedido_Front"))))
                     .withColumn("ValorEstorno", when(col("ValorEstorno") == col("ValorPagoPeloCliente"), "Total").otherwise("Parcial"))                   
                     .drop(df_CompraEntregaStatusEstorno.IdCompra)
                     .drop(df_CompraFormaPagamento.IdCompra)
                     .drop(df_CompraFormaPagamento.IdFormaPagamento)
                     .drop(df_PagamentoCliente.IdCompra)
                     .drop(df_CompraCliente.IdCompra)
                     .drop(df_Estorno.Pedido)
                     .drop(df_Compra.IdCompra)
                     .select("Id", "Nome", "Email", "TelefoneResidencial", "TelefoneCelular", "Loja", "Estorno", "Nome_Cartao", "Tipo_Estorno", "Pedido", col("ValorEstorno").alias("Restituicao"), "dataultimaatualizacao").filter(col("IdTipoPagamento") == "2"))

In [0]:
colunas = df_Front.drop("dataultimaatualizacao", "data_minima").columns
colunas.extend(df_Vale.drop("email", "nome_cartao", "tipo_estorno", "telefonecelular", "id", "Estorno", "restituicao", "pedido", "nome", "loja", "telefoneresidencial", "dataultimaatualizacao", "data_minima").columns)

In [0]:
df_RestituicaoConfirmada = df_Front.select(colunas).unionByName(df_Vale.select(colunas))

In [0]:
df_RestituicaoConfirmada = df_RestituicaoConfirmada.dropDuplicates()

In [0]:
df_RestituicaoConfirmada.repartition(1).write.format("csv").option("sep", ";").option("header","true").mode("overwrite").save("/mnt/blob/landingdev/Leonardo_Andrade/Restituicao_Confirmada")

In [0]:
%run MotorIngestao/RAW/DW_Connector

In [0]:
writeDW(spark, df_RestituicaoConfirmada, 'gestao_cliente.restituicao_confirmada', mode='overwrite', additional_options=None)